In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Splitting the dataframes

In [ ]:
transaction = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
customer = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')


In [ ]:
#Checking the number of rows 
transaction.loc[transaction['customer_id']=='00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'].shape

# Join transaction and customer by customer id and get age and postal code

In [ ]:
customer_detail_by_trans = pd.merge(transaction,customer,on='customer_id')

# Let us now merge the table with article table

In [ ]:
customer_detail_by_trans_by_article = pd.merge(customer_detail_by_trans,articles[['article_id','prod_name','product_type_name','product_group_name','graphical_appearance_name','colour_group_name','perceived_colour_value_name','perceived_colour_master_name','department_name','index_name','index_group_name','section_name','garment_group_name','detail_desc']],on='article_id')

# Table below has no identifiers but more of names for features

In [ ]:
customer_detail_by_trans_by_article.head(5)

# Generate word cloud on a dataframe column "detail_desc"

In [ ]:
#Final word cloud after all the cleaning and pre-processing
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
comment_words = ' '
stopwords = set(STOPWORDS) 

In [ ]:
# iterate through the csv file 
for val in articles['detail_desc']: 

   # typecaste each val to string 
   val = str(val) 

   # split the value 
   tokens = val.split() 

# Converts each token into lowercase 
for i in range(len(tokens)): 
    tokens[i] = tokens[i].lower() 

for words in tokens: 
    comment_words = comment_words + words + ' '


wordcloud = WordCloud(width = 800, height = 800, 
            background_color ='white', 
            stopwords = stopwords, 
            min_font_size = 10).generate(comment_words) 

# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 

### We could see from the above cell most people prefer soft and cotton items

In [ ]:
customer['age'].hist(bins=10)

# Let us first define what is expensive

In [ ]:
customer_prod_price = pd.merge(articles[['article_id','product_group_name']],transaction[['article_id','price']],on='article_id').drop_duplicates()
customer_prod_price = customer_prod_price[['product_group_name','price']]
customer_prod_price.plot.scatter(x = 'product_group_name', y = 'price', s = 'price',figsize=(30, 15), c = 'red');

In [ ]:
customer_prod_price['product_group_name'].unique()

# How fashion news influences the buying behaviour 

In [ ]:
fashion_news_group = pd.merge(customer[['customer_id','fashion_news_frequency']],transaction[['customer_id','price']],on='customer_id').drop_duplicates()
fashion_news_group = fashion_news_group[['fashion_news_frequency','price']]
fashion_news_group['fashion_news_frequency'] = fashion_news_group['fashion_news_frequency'].replace(np.nan, 'None')
fashion_news_group['fashion_news_frequency'] = fashion_news_group['fashion_news_frequency'].replace('NONE', 'None')
fashion_news_group.plot.scatter(x = 'fashion_news_frequency', y = 'price', s = 'price',figsize=(15, 15), c = 'red');

In [ ]:
import seaborn as sns
# Plotting a bar graph of the number of stores in each city, for the first ten cities listed
# in the column 'City'
sales_channel  = transaction['sales_channel_id'].value_counts()
sales_channel_count = sales_channel[:3,]
plt.figure(figsize=(10,5))
sns.barplot(sales_channel_count.index, sales_channel_count.values, alpha=0.8)
plt.title('Sales Channel split')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Channel id', fontsize=12)
plt.show()
